In [1]:
#import of dependencies
import numpy as np                            #Library that has extensive set of math methods and functions.
from scipy.integrate import odeint            #Library that has function for solving differentail equations
from scipy.interpolate import interp1d        #Library containing fuctions of interpolating data
from scipy.optimize import curve_fit          #Library containing fuctions of fitting paramaters to data
from scipy.stats import t as tdist
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import math
from itertools import repeat
from datetime import timedelta
import csv

#Library to plot data
import plotly.graph_objects as go


#Imports data from project github
mobility_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/SwedenMobilityData.csv')
admission_data = pd.read_json('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/Inskrivningsdata.json')
VT_mobility_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/VT_mobility.csv')

In [2]:
# A array of time points in days for the model to use
prediction_time = 173
t = np.linspace(1, prediction_time, prediction_time)

#Defines how many data point are to be used for model fitting
fittingPoints = 173 # Max 173 days
t_fittingdata = np.linspace(0, fittingPoints, fittingPoints)
startdata = 42  #number of intial data points for MAP/ISalpha calculation in days


dailyAdmissions = admission_data['Inskrivningar']


In [3]:
#interpolates the weekly Västtrafik data to daily data
#then creates a mobility function
temp_VT = VT_mobility_data['Totalt']

T = []
for week in range(len(temp_VT)-1):
    T.extend(np.linspace(temp_VT[week],temp_VT[week+1],8)[:7])
    
VT_mobilityfunc = interp1d(t,T[:prediction_time],kind='linear',fill_value="extrapolate")

In [4]:
#Weiss implementation

# Total population, N.
N = 700000 #Gothenburg city
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0

tau = 1         #transmisability chance
gamma =  0.2    #recovery rate 1/days
lag = 21        #Lag from infection to admission


# Initial conditions vector
y0 = S0, I0, R0
p = 0.013 #fraction of the present number of infectious individuals

def modelfunction(t,c0,c1): #Enbedded function that takes 
  def betafunc(t):
    kappa =  c0+c1*VT_mobilityfunc(t)
    b = kappa*tau
    return b/N

  # The SIR model differential equations.
  def deriv(y, t, N):
    S, I, R = y
    dSdt = -betafunc(t-lag) * S * I 
    dIdt = betafunc(t-lag) * S * I - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt
  
  result = odeint(deriv, y0, t, args=(N,))
  S, I, R = result.T
  #Returns total admissions for each day
  return I*p

#Curve fit function that takes the modelfunction with the given parameters to optimize and returns them with optimal value and covariance
popt, pcov = curve_fit(modelfunction, t_fittingdata, dailyAdmissions[:fittingPoints], p0=[0.4198,0.003781])

#best VT guess p0=[0.4198,0.003781])
#Best google guess p0=[3,0.6])

#plots model against admissions data
fig = go.Figure() 
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=fittingPoints)), 
                    y=dailyAdmissions,
                    mode='markers',
                    name='Admission data'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=modelfunction(t,*popt),
                    mode='lines',
                    name='Predicted admissions'))
fig.update_layout(
                    title='Time dependant SIR model',
                    xaxis_title='Weeks',
                    yaxis_title='Admissions',
                    autosize=False,
                    width=1000,
                    
                    font=dict(
                    family="Courier New, monospace",
                    size=14,
                    color="RebeccaPurple"
    )
)
fig.show()

In [5]:
numBetas = 50 #number of betas used for normal dist for conf. intervalls
numEpsilon = 500

##################  Confidence intervall calculation
confint = np.zeros([prediction_time,2]) #defines confidence intervall arrays


# Generates an array of betas using normal dist.
betas=np.random.multivariate_normal(popt,pcov,numBetas) 
values = np.zeros((prediction_time,numBetas))

#Evalutes the model function for each given beta
for beta in range(len(betas)): 
  values[:,beta] = (modelfunction(t,*betas[beta]))

#Calculates the confidence intervals for each time step  
for day in range(len(t)):
  confint[day,0]=np.quantile(values[day,:],0.025)
  confint[day,1]=np.quantile(values[day,:],0.975)
  

##################  Prediction intervall calculation

#calcuales standard deviation of residual between model and data
std_residual = np.std(modelfunction(t_fittingdata,*popt)-dailyAdmissions[:fittingPoints], ddof=2) #calcuales standard deviation of residual between model and data
predint = np.zeros([prediction_time,2])

#init of arrays/matrix
temp = []
pred_matrix = np.zeros((prediction_time,(numBetas*numEpsilon)))

#loops trough model values each day a adds the error values generated from t-dist to 
#each prediction and returns the prediction matrix
for day in range(prediction_time): 
  for beta in range(len(betas)):
      temp =  np.add(modelfunction(t,*betas[beta])[day],tdist.rvs(prediction_time,0,std_residual,numEpsilon))
      pred_matrix[day,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

#calculates the prediction intervals for each time step
for day in range(prediction_time): #calculates the prediction intervals
    predint[day,0] = np.quantile(pred_matrix[day],0.025)
    predint[day,1] = np.quantile(pred_matrix[day],0.975)
     

fig = go.Figure() 
#plots the fitting admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=fittingPoints)), 
                    y=dailyAdmissions,
                    mode='markers',
                    name='Inskrivningsdata'))
#plots all the available admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=fittingPoints), freq="D", periods=prediction_time-fittingPoints)),
                    y=dailyAdmissions[fittingPoints:],
                    mode='markers',
                    marker=dict(
                        color='firebrick',
                    ),
                    name='Admission data'))
#plots model admissions
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=modelfunction(t,*popt),
                    mode='lines',
                    name='predikterade inskrivningar'))
#plots lower confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,0],
                    mode='lines',
                    name='Undre konfidensintervall',
                    line = dict( width=2, dash='dash')))
#plots Upper confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,1],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
#plots lower prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,0],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots upper prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,1],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots RMSE annotation box
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(dailyAdmissions[:prediction_time],modelfunction(t,*popt), squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Västtrafik mobilitets baserad SIR modell',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=prediction_time)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()

Cell calcualtes the cumulative admissions and confidens/prediktion intervalls

In [6]:
numBetas = 50 #number of betas used for normal dist for conf. intervalls
numEpsilon = 500

##################  Confidence intervall calculation
#defines confidence intervall arrays
confint = np.zeros([prediction_time,2]) 


# Generates an array of betas using normal dist.
betas=np.random.multivariate_normal(popt,pcov,numBetas) 
values = np.zeros((prediction_time,numBetas))

# Generates an array of betas using normal dist.
for beta in range(len(betas)): 
  values[:,beta] = np.cumsum(modelfunction(t,*betas[beta]))
  
#Evaluates the model function for each given beta
for day in range(len(t)):
  confint[day,0]=np.quantile(values[day,:],0.025)
  confint[day,1]=np.quantile(values[day,:],0.975)
  

##################  Prediction intervall calculation
#calcuales standard deviation of residual between model and data
std_residual = np.std(modelfunction(t_fittingdata,*popt)-dailyAdmissions[:fittingPoints], ddof=2) #calcuales standard deviation of residual between model and data
predint = np.zeros([prediction_time,2])

temp = []
pred_matrix = np.zeros((prediction_time,(numBetas*numEpsilon)))

#loops trough model values each day a adds the error values generated from t-dist to each prediction 
#and returns the prediction matrix
for day in range(prediction_time): #loops trough model values each day a adds the error values generated from t-dist to each prediction 
  for beta in range(len(betas)):
      temp =  np.add(np.cumsum(modelfunction(t,*betas[beta]))[day],tdist.rvs(prediction_time,0,std_residual,numEpsilon))
      pred_matrix[day,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

#calculates the prediction intervals for each time step
for day in range(prediction_time): #calculates the prediction intervals
    predint[day,0] = np.quantile(pred_matrix[day],0.025)
    predint[day,1] = np.quantile(pred_matrix[day],0.975)

fig = go.Figure()
#plots the fitting admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=fittingPoints)), 
                    y=np.cumsum(dailyAdmissions),
                    mode='markers',
                    name='Inskrivningsdata'))
#plots all the available admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=fittingPoints), freq="D", periods=prediction_time-fittingPoints)),
                    y=np.cumsum(dailyAdmissions)[fittingPoints:],
                    mode='markers',
                    marker=dict(
                        color='firebrick',
                    ),
                    name='Admission data'))
#plots model admissions
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=np.cumsum(modelfunction(t,*popt)),
                    mode='lines',
                    name='predikterade kumulativa inskrivningar'))
#plots lower confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,0],
                    mode='lines',
                    name='Undre konfidensintervall',
                    line = dict( width=2, dash='dash')))
#plots Upper confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,1],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
#plots lower prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,0],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots upper prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,1],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots RMSE annotation box
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(dailyAdmissions[:prediction_time],modelfunction(t,*popt), squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Västtrafik mobilitets baserad SIR modell',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=prediction_time)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()

In [7]:
#calcualtes MAPE for 21 days ahead of the fitting data as it increases with time

#header for csv file
header = ['Idex', 'Fitting points', 'MAPE']
RMSEovertime = []   #init RMSE array


#writes a csv file of Index, number of fitting points and MAP
with open('SIR_MAPE_daily.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    #loops over all data point for fitting and curve fits the model and calculates MAPE
    for day in range(len(dailyAdmissions)):
        #breaks loop when reaching final value
        if day+startdata+21 == len(dailyAdmissions):
            break
        
        #defines time arrays used for calculation
        t_MAPE1 = np.linspace(0,day+startdata,day+startdata)
        t_MAPE2 = np.linspace(0,day+startdata+21,day+startdata+21)

        #curve fits model to fitting data
        MAPE_popt, pcov = curve_fit(modelfunction, t_MAPE1, dailyAdmissions[:day+startdata], p0=[0.4198,0.003781])

        temp_RMSEovertime = [day, day + startdata - 21, mean_absolute_percentage_error(np.cumsum(dailyAdmissions[: day + startdata + 21])[-21:], np.cumsum(modelfunction(t_MAPE2, *MAPE_popt))[-21:])]

        #Appends matrix of all MAP vallues
        RMSEovertime.append(temp_RMSEovertime)

        writer.writerow(temp_RMSEovertime)

In [8]:
numBetas = 20 #number of betas used for normal dist for conf. intervalls
numEpsilon = 200
alpha = 0.05
ISalpha_score = []

#header for csv file
header = ['Idex', 'Fitting points', 'ISalpha'] 

#writes a csv file of Index, number of fitting points and ISalpha
with open('SIR_ISalpha_daily.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    #loops trough the weeks as the admission data increases and returns the avarge ISalpha
    #score for 21 days at a time
    for day in range(len(dailyAdmissions)):
        #breaks loop when reaching final value
        if day+startdata+21 > len(dailyAdmissions):
            break
        t_ISalpha1 = np.linspace(0,day+startdata,day+startdata)       #used for fitting
        t_ISalpha2 = np.linspace(0,day+startdata+21,day+startdata+21)     #used for residual calculation
        
        #curve fits model to fitting data
        ISalpha_popt, ISalpha_pcov = curve_fit(modelfunction, t_ISalpha1, dailyAdmissions[:day+startdata], p0=[0.4198,0.003781])
        
        #calcuales standard deviation of residual between model and data using the fitting data + 3 weeks forward
        std_residual_ISalpha = np.std(modelfunction(t_ISalpha1,*ISalpha_popt)-dailyAdmissions[:day+startdata], ddof=2)
        # Generates an array of betas using normal dist.
        ISalpha_betas=np.random.multivariate_normal(ISalpha_popt,ISalpha_pcov,numBetas)  
        
        #Generates the prediction matrix for the prediction interval
        predint_ISalpha = np.zeros([21,3])
        
        temp = []
        pred_matrix = np.zeros((21,(numBetas*numEpsilon)))
        
        #loops trough model values each week a adds the error values generated from t-dist to each prediction 
        for dayInInterval in range(21): 
            for beta in range(len(ISalpha_betas)):
                temp =  np.add(np.cumsum(modelfunction(t_ISalpha2,*ISalpha_betas[beta]))[day+startdata+dayInInterval],tdist.rvs(dayInInterval+startdata+3,0,std_residual_ISalpha,numEpsilon))
                pred_matrix[dayInInterval,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

        #calculates the prediction intervals
        for dayInInterval in range(21): #calculates the prediction intervals
            predint_ISalpha[dayInInterval,0] = np.quantile(pred_matrix[dayInInterval],0.025)
            predint_ISalpha[dayInInterval,1] = np.quantile(pred_matrix[dayInInterval],0.975)
            predint_ISalpha[:,2] = np.cumsum(modelfunction(t_ISalpha2,*ISalpha_betas[beta]))[-21:]
        
        #predint_ISalpha now contains lower, upper and the model output (columms) for the 3 weeks (rows)
        
        temp_score = []

        #calculates the score for each week of the 3 week interval and then avrages them
        for dayInInterval,value in enumerate(predint_ISalpha):

            u = predint_ISalpha[dayInInterval,0]
            l = predint_ISalpha[dayInInterval,1]
            y = predint_ISalpha[dayInInterval,2]

            coeff1 = 1 if y < l else 0
            coeff2 = 1 if y > u else 0

            #calcualtes the ISalpha score 
            temp_score.append((u-l)+2/alpha*(l-y)*coeff1+2/alpha*(y-u)*coeff2)

        temp_ISalpha_score = [day+startdata-21, np.average(temp_score)]
        ISalpha_score.append(temp_ISalpha_score)
        
        # write the data
        writer.writerow(temp_ISalpha_score)

In [9]:
#calcualtes the cumulative admissions at the end date as the fitting points increase
#header for csv file
header = ['Fitting points', 'Total admissions'] 
Totaladmissionsovertime = []


#writes a csv file of Index, number of fitting points and total admissions
with open('Totaladmissions_daily.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    
    #loops over all data point for fitting and curve fits the model and calculates MAPE
    for day in range(len(dailyAdmissions)):
        #breaks loop when reaching final value
        if day+startdata+3 > len(dailyAdmissions):
            break
        
        #inits temp array for writing to csv file and time array for model
        temp_totaladmissions = []
        t_totaladmissions = np.linspace(0,day+startdata,day+startdata)
        
        #curve fits model to fitting data
        TotalAdmissions_popt, TotalAdmissions_covt = curve_fit(modelfunction, t_totaladmissions, dailyAdmissions[:day+startdata], p0=[0.4198,0.003781])
        
        #creates temp list of Index, number of fitting points and MAP
        temp_totaladmissions = [day+startdata-21,np.cumsum(modelfunction(t,*TotalAdmissions_popt))[-1]]
        
        
        #Appends matrix of all MAP vallues
        Totaladmissionsovertime.append(temp_totaladmissions)
        
        # write the data
        writer.writerow(temp_totaladmissions)

Cell that seatches for a global minimum of the paramaters given using RMSE

In [10]:
#VT global param search
paramC0 = np.linspace(0,4,50)  #x-values
paramC1 = np.linspace(0,1,50)  #y-values

# #Google global param search
# params.append(np.linspace(0,4,50)) 
# params.append(np.linspace(0,1,50))

#Loops trough parameter room and evaluates RMSE of the model function
result = np.zeros((len(paramC0),len(paramC1)))
for param1 in range(len(paramC0)):
    for param0 in range(len(paramC1)):
        result[param0,param1] = math.log(mean_squared_error(dailyAdmissions,modelfunction(t,paramC0[param0],paramC1[param1]), squared = False))


fig = go.Figure() 
#plots a contour plot of the paramter room against the RMSE of the model function
fig.add_contour(
                    z=result,
                    x=paramC0,
                    y=paramC1,
                    colorbar=dict(
                        titleside='right',
                        title='Log RMSE', # title here
                    )
)  
  

fig.update_layout(
                    title='Global parameter search Google mobility data',
                    xaxis_title='c0 value',
                    yaxis_title='c1 value',
                    autosize=False,
                    width=800,
                    height=800,
                    font=dict(
                        family="Courier New, monospace",
                        size=18,
                        color="RebeccaPurple"
    )
)
fig.show()
